# Visualizing Molecular Structures in VR

Download real protein structures from the Protein Data Bank and explore them in virtual reality.

**What you'll create:** Interactive 3D visualizations of proteins with atoms colored by element type.

In [ ]:
# Auto-install required packages
import sys
import subprocess
import importlib

try:
    import immersivepoints as ip
except ImportError:
    print("Installing immersivepoints...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'immersivepoints', '--quiet'])
    import site
    importlib.reload(site)
    import immersivepoints as ip

import numpy as np
import requests
from io import StringIO

try:
    from Bio.PDB import PDBParser
except ImportError:
    print("Installing biopython...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'biopython', '--quiet'])
    importlib.reload(site)
    from Bio.PDB import PDBParser

print("✓ Ready to visualize proteins!")

## Step 1: Download a Protein from PDB

Choose from these interesting proteins:
- `1A3N` - **Hemoglobin** (oxygen transport in blood)
- `1MSO` - **Insulin** (blood sugar regulation)
- `1BNA` - **DNA double helix**
- `6VXX` - **SARS-CoV-2 spike protein**
- `2HHB` - **Deoxyhemoglobin**
- `1AKE` - **Adenylate kinase** (enzyme)

In [ ]:
def download_pdb(pdb_id):
    """Download PDB file from RCSB database."""
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise ValueError(f"Could not download PDB {pdb_id}")

# Choose your protein here
PDB_ID = "1A3N"  # Hemoglobin

print(f"Downloading {PDB_ID} from RCSB Protein Data Bank...")
pdb_data = download_pdb(PDB_ID)
print(f"✓ Downloaded {len(pdb_data):,} bytes")

## Step 2: Parse Atomic Coordinates

Extract atom positions and assign colors based on element type.

In [ ]:
def element_to_hue(element):
    """Map chemical element to color (hue 0-1)."""
    colors = {
        'C': 0.0,    # Carbon - gray
        'N': 0.6,    # Nitrogen - blue
        'O': 0.0,    # Oxygen - red
        'S': 0.15,   # Sulfur - yellow
        'P': 0.08,   # Phosphorus - orange
        'H': 0.5,    # Hydrogen - white
        'FE': 0.95,  # Iron - brown
        'ZN': 0.5,   # Zinc - gray
        'MG': 0.33,  # Magnesium - green
    }
    return colors.get(element.upper(), 0.5)

def pdb_to_points(pdb_string):
    """Convert PDB file to point cloud."""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', StringIO(pdb_string))
    
    points = []
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coord = atom.get_coord()
                    hue = element_to_hue(atom.element)
                    points.append([coord[0], coord[1], coord[2], hue])
    
    points = np.array(points, dtype=np.float32)
    
    # Center the molecule
    points[:, :3] -= points[:, :3].mean(axis=0)
    
    # Scale for VR
    points[:, :3] *= 0.1
    
    return points

print("Parsing PDB file...")
points = pdb_to_points(pdb_data)
print(f"✓ Extracted {len(points):,} atoms")
print(f"  Range: X=[{points[:,0].min():.1f}, {points[:,0].max():.1f}] "
      f"Y=[{points[:,1].min():.1f}, {points[:,1].max():.1f}] "
      f"Z=[{points[:,2].min():.1f}, {points[:,2].max():.1f}]")

## Step 3: Visualize Inline

Preview your molecule right here in the notebook!

In [ ]:
# Render inline - you can rotate and zoom
ip.renderPoints(points, point_size=0.3, background_color=0x000000, width=800, height=600)

## Step 4: Generate VR Link

Create a link to view in your VR headset!

In [ ]:
ip.showVR(points, point_size=0.3)

## Step 5: Save for Upload (Optional)

Save the file to upload to immersivepoints.com later.

In [ ]:
output_file = f"{PDB_ID}_molecule.xyzi"
points.astype(np.float32).byteswap().tofile(output_file)
print(f"✓ Saved to {output_file}")
print(f"  File size: {len(points) * 16 / 1024:.1f} KB")
print(f"\n📤 Upload at: https://immersivepoints.com/upload.html")

---

## Try Different Proteins

Change `PDB_ID` above and re-run to explore:

**Antibodies & Immune System:**
- `1IGT` - Immunoglobulin
- `1HZH` - Antibody complex

**Enzymes:**
- `4HHB` - Hemoglobin variant
- `1TRZ` - DNA polymerase

**DNA/RNA:**
- `1EHZ` - RNA structure

**Viral Proteins:**
- `6VXX` - COVID-19 spike protein
- `1JSF` - Influenza hemagglutinin

Browse 200,000+ structures at **https://www.rcsb.org/**